In [20]:
# Importing dependencies
import pandas as pd
import numpy as np
from datetime import datetime,date

import warnings
warnings.filterwarnings('ignore')

In [21]:
csv = ('Resources/DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
nyc_inspect = pd.read_csv(csv)

## 1. Initial exploration of dataset

In [23]:
nyc_inspect.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,06/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,30.0,NaN,NaN,08/28/2017,Cycle Inspection / Initial Inspection
1,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.,Not Critical,NaN,NaN,NaN,08/28/2017,Administrative Miscellaneous / Initial Inspection
2,50046354,VITE BAR,QUEENS,2507,BROADWAY,11106.0,3478134702,Italian,10/03/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2.0,NaN,NaN,08/28/2017,Pre-permit (Operational) / Initial Inspection
3,50061389,TACK'S CHINESE TAKE OUT,STATEN ISLAND,11C,HOLDEN BLVD,10314.0,7189839854,Chinese,05/17/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,46.0,NaN,NaN,08/28/2017,Pre-permit (Operational) / Initial Inspection
4,41516263,NO QUARTER,BROOKLYN,8015,5 AVENUE,11209.0,7187019180,American,03/30/2017,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,18.0,NaN,NaN,08/28/2017,Cycle Inspection / Initial Inspection


In [24]:
# Finding range of dates for all reports
max_date = nyc_inspect['RECORD DATE'].max()
min_date = nyc_inspect['RECORD DATE'].min()
print(f'Dataset starts from {min_date} to {max_date}')

Dataset starts from 08/28/2017 to 08/28/2017


In [25]:
# Determing the number of rows and columns in dataset
row_col = nyc_inspect.shape
print(f'Dataset contains {row_col[0]} rows and {row_col[1]} columns')


Dataset contains 399918 rows and 18 columns


In [30]:
nyc_inspect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399918 entries, 0 to 399917
Data columns (total 18 columns):
CAMIS                    399918 non-null int64
DBA                      399559 non-null object
BORO                     399918 non-null object
BUILDING                 399809 non-null object
STREET                   399909 non-null object
ZIPCODE                  399909 non-null float64
PHONE                    399913 non-null object
CUISINE DESCRIPTION      399918 non-null object
INSPECTION DATE          399918 non-null object
ACTION                   398783 non-null object
VIOLATION CODE           393414 non-null object
VIOLATION DESCRIPTION    392939 non-null object
CRITICAL FLAG            399918 non-null object
SCORE                    376704 non-null float64
GRADE                    195413 non-null object
GRADE DATE               192820 non-null object
RECORD DATE              399918 non-null object
INSPECTION TYPE          398783 non-null object
dtypes: float64(2), int6

In [26]:
# Determing labels for each column
nyc_inspect.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE'],
      dtype='object')

In [27]:
nyc_inspect.describe()

,CAMIS,ZIPCODE,SCORE
count,3.999180e+05,399909.000000,376704.000000
mean,4.441662e+07,10673.386933,18.910181
std,4.245117e+06,598.959356,12.958955
min,3.007544e+07,10001.000000,-2.000000
25%,4.121603e+07,10022.000000,11.000000
50%,4.160911e+07,10467.000000,15.000000
75%,5.000934e+07,11229.000000,24.000000
max,5.006908e+07,11697.000000,151.000000


In [28]:
nyc_inspect.isna().sum()

CAMIS                         0
DBA                         359
BORO                          0
BUILDING                    109
STREET                        9
ZIPCODE                       9
PHONE                         5
CUISINE DESCRIPTION           0
INSPECTION DATE               0
ACTION                     1135
VIOLATION CODE             6504
VIOLATION DESCRIPTION      6979
CRITICAL FLAG                 0
SCORE                     23214
GRADE                    204505
GRADE DATE               207098
RECORD DATE                   0
INSPECTION TYPE            1135
dtype: int64

Columns of interest from rat_data:

- 'Sighting Date', 
- 'Incident',
- 'Incident Zip', 
- 'Borough',
- 'X Coordinate (State Plane)', 
- 'Y Coordinate (State Plane)', 
- 'Latitude',
- 'Longitude'

Based on this, we're interested in the following columns in the nyc inspection data:

- 'CAMIS': violation ID
- 'BORO': Name of borough where violation occured
- 'ZIPCODE': Zip code where violation occured
- 'INSPECTION DATE': self-explanatory
- 'VIOLATION DESCRIPTION': self-explanatory
- 'CRITICAL FLAG': severity of violation

**Also we'll remove all rows with missing or incorrectly formatted values**

## 2. Data cleaning

In [35]:
# Reformatting date
nyc_inspect['INSPECTION DATE'] = pd.to_datetime(nyc_inspect['INSPECTION DATE']).dt.date

In [37]:
# Removing unecessary columns
nyc_inspect = nyc_inspect.drop(['DBA', 
                         'BUILDING', 
                         'STREET', 
                         'PHONE', 
                         'CUISINE DESCRIPTION', 
                         'ACTION', 
                         'VIOLATION CODE', 
                         'SCORE', 
                         'GRADE',
                         'GRADE DATE', 
                         'RECORD DATE', 
                         'INSPECTION TYPE'], 
                         axis=1)

# Renaming columns for clarity
nyc_inspect.rename(columns={'CAMIS': 'Violation ID', 
                         'BORO': 'Borough', 
                         'ZIPCODE': 'Incident Zip', 
                         'INSPECTION DATE': 'Violation Date', 
                         'VIOLATION DESCRIPTION': 'Violation Description', 
                         'CRITICAL FLAG': 'Critical Flag'}, 
                         inplace=True)

nyc_inspect.head()

,Violation ID,Borough,Incident Zip,Violation Date,Violation Description,Critical Flag
0,40511702,MANHATTAN,10016.0,2015-06-15,Hot food item not held at or above 140Âº F.,Critical
1,40511702,MANHATTAN,10016.0,2014-11-25,Current letter grade card not posted.,Not Critical
2,50046354,QUEENS,11106.0,2016-10-03,Non-food contact surface improperly constructe...,Not Critical
3,50061389,STATEN ISLAND,10314.0,2017-05-17,Cold food item held above 41Âº F (smoked fish ...,Critical
4,41516263,BROOKLYN,11209.0,2017-03-30,Live roaches present in facility's food and/or...,Critical


In [44]:
# Finding mentions of rodents
rat_violations = nyc_inspect[nyc_inspect['Violation Description'].str.contains("rat")==True].reset_index()

In [45]:
rat_violations.head()

,index,Violation ID,Borough,Incident Zip,Violation Date,Violation Description,Critical Flag
0,3,50061389,STATEN ISLAND,10314.0,2017-05-17,Cold food item held above 41Âº F (smoked fish ...,Critical
1,6,50058069,BROOKLYN,11215.0,2017-06-22,Evidence of rats or live rats present in facil...,Critical
2,7,40807238,STATEN ISLAND,10304.0,2017-06-14,Cold food item held above 41Âº F (smoked fish ...,Critical
3,8,41547684,STATEN ISLAND,10310.0,2015-03-10,Cold food item held above 41Âº F (smoked fish ...,Critical
4,9,40376944,MANHATTAN,10012.0,2015-10-06,Cold food item held above 41Âº F (smoked fish ...,Critical


In [59]:
# Checking for missing values
rat_violations.isna().sum()

index                    0
Violation ID             0
Borough                  0
Incident Zip             0
Violation Date           0
Violation Description    0
Critical Flag            0
dtype: int64

In [52]:
# Reformatting borough names
# Reformattiing zip column

rat_violations['Borough'] = rat_violations['Borough'].str.lower()
rat_violations['Borough'] = rat_violations['Borough'].str.title()

rat_violations['Incident Zip'] = rat_violations['Incident Zip'].round(0).astype(int)

rat_violations.head()

,index,Violation ID,Borough,Incident Zip,Violation Date,Violation Description,Critical Flag
0,3,50061389,Staten Island,10314,2017-05-17,Cold food item held above 41Âº F (smoked fish ...,Critical
1,6,50058069,Brooklyn,11215,2017-06-22,Evidence of rats or live rats present in facil...,Critical
2,7,40807238,Staten Island,10304,2017-06-14,Cold food item held above 41Âº F (smoked fish ...,Critical
3,8,41547684,Staten Island,10310,2015-03-10,Cold food item held above 41Âº F (smoked fish ...,Critical
4,9,40376944,Manhattan,10012,2015-10-06,Cold food item held above 41Âº F (smoked fish ...,Critical


In [57]:
min_date = rat_violations['Violation Date'].min()
max_date = rat_violations['Violation Date'].max()

row_col = rat_violations.shape

# Description of cleaned dataset we'll use for our analysis

print('Cleaned NYC Restaurant Rat Violations')
print('-----------------------')
print(f'Table contains {row_col[0]} rows and {row_col[1]} columns')
print(f'Dates range from {min_date} to {max_date}')

Cleaned NYC Restaurant Rat Violations
-----------------------
Table contains 100892 rows and 7 columns
Dates range from 2012-06-23 to 2017-08-26


In [58]:
# Saving final csv for analysis
rat_violations.to_csv(r'clean_csv/final_rat_violations.csv')